<a href="https://colab.research.google.com/github/vivekmithlesh/GSOC_MIC_TEST/blob/main/Crisis_Geolocation_%26_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install geopy folium geotext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
from folium.plugins import HeatMap
from geotext import GeoText
import time

In [5]:
from google.colab import files
uploaded = files.upload()

Saving task2_sentiment_risk.csv to task2_sentiment_risk.csv


In [6]:
df = pd.read_csv("task2_sentiment_risk.csv")
df.head()

,post_id,timestamp,content,cleaned_content,likes,retweets,sentiment,risk_level
0,1909293034514063580,2025-04-07 17:11:47+00:00,Portnoy is hopeless. He would rather lose 10 t...,portnoy hopeless would rather lose 10 15 portf...,0,0,Negative,High-Risk
1,1909293033876599291,2025-04-07 17:11:46+00:00,It’s hard to care about all this shit cus it’s...,hard care shit cus damn near monthly something...,0,0,Negative,Moderate Concern
2,1909293033004450050,2025-04-07 17:11:46+00:00,Suddenly I’m not depressed anymore https://t.c...,suddenly im depressed anymore,0,0,Negative,Moderate Concern
3,1909293029967478936,2025-04-07 17:11:45+00:00,"According to common sense, the theory goes fro...",according common sense theory goes verificatio...,0,0,Negative,Low Concern
4,1909293028759527575,2025-04-07 17:11:45+00:00,RT @Bubblebathgirl: Unhinged leftist protester...,rt bubblebathgirl unhinged leftist protester s...,0,155,Neutral,Low Concern


In [7]:
def extract_location(text):
    places = GeoText(str(text))
    if places.cities:
        return places.cities[0]
    elif places.countries:
        return places.countries[0]
    else:
        return None

df['extracted_location'] = df['cleaned_content'].apply(extract_location)
df['extracted_location'].dropna().head(10)

,extracted_location


In [8]:
geolocator = Nominatim(user_agent="gsoc_map")

def safe_geocode(location):
    try:
        if pd.isna(location):
            return None, None
        loc = geolocator.geocode(location, timeout=10)
        time.sleep(1)  # be polite with the API
        if loc:
            return loc.latitude, loc.longitude
    except GeocoderTimedOut:
        return safe_geocode(location)
    return None, None

df[['lat', 'lon']] = df['extracted_location'].apply(lambda loc: pd.Series(safe_geocode(loc)))
df = df.dropna(subset=['lat', 'lon'])
df[['extracted_location', 'lat', 'lon']].head()

,extracted_location,lat,lon


In [11]:
for i, row in df.iterrows():
    print(f"Text: {row['cleaned_content']}")
    print(f"Cities: {GeoText(str(row['cleaned_content'])).cities}")
    print(f"Countries: {GeoText(str(row['cleaned_content'])).countries}")
    print("-" * 50)

In [12]:
import pandas as pd

data = {
    'cleaned_content': [
        "I feel so lost and hopeless in Delhi",
        "Need urgent mental help in Mumbai please",
        "My anxiety is killing me, anyone in Bangalore to talk?",
        "I feel like giving up... stuck in Chennai",
        "I'm overwhelmed and scared, living in Kolkata"
    ]
}

df = pd.DataFrame(data)
df.head()

,cleaned_content
0,I feel so lost and hopeless in Delhi
1,Need urgent mental help in Mumbai please
2,"My anxiety is killing me, anyone in Bangalore ..."
3,I feel like giving up... stuck in Chennai
4,"I'm overwhelmed and scared, living in Kolkata"


In [13]:
from geotext import GeoText

def extract_location(text):
    places = GeoText(str(text))
    if places.cities:
        return places.cities[0]
    elif places.countries:
        return places.countries[0]
    else:
        return None

df['extracted_location'] = df['cleaned_content'].apply(extract_location)
df

,cleaned_content,extracted_location
0,I feel so lost and hopeless in Delhi,Delhi
1,Need urgent mental help in Mumbai please,Mumbai
2,"My anxiety is killing me, anyone in Bangalore ...",Bangalore
3,I feel like giving up... stuck in Chennai,Chennai
4,"I'm overwhelmed and scared, living in Kolkata",Kolkata


In [14]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

geolocator = Nominatim(user_agent="gsoc_mapper")

def safe_geocode(location):
    try:
        if pd.isna(location):
            return None, None
        loc = geolocator.geocode(location, timeout=10)
        time.sleep(1)
        if loc:
            return loc.latitude, loc.longitude
    except GeocoderTimedOut:
        return safe_geocode(location)
    return None, None

df[['lat', 'lon']] = df['extracted_location'].apply(lambda loc: pd.Series(safe_geocode(loc)))
df = df.dropna(subset=['lat', 'lon'])
df

,cleaned_content,extracted_location,lat,lon
0,I feel so lost and hopeless in Delhi,Delhi,28.627393,77.171695
1,Need urgent mental help in Mumbai please,Mumbai,19.054999,72.869203
2,"My anxiety is killing me, anyone in Bangalore ...",Bangalore,12.988157,77.622600
3,I feel like giving up... stuck in Chennai,Chennai,13.083694,80.270186
4,"I'm overwhelmed and scared, living in Kolkata",Kolkata,22.572646,88.363895


In [15]:
import folium
from folium.plugins import HeatMap

heat_data = [[row['lat'], row['lon']] for index, row in df.iterrows()]

m = folium.Map(location=[20.5937, 78.9629], zoom_start=4)
HeatMap(heat_data).add_to(m)
m.save("crisis_heatmap.html")
m

In [16]:
top_locations = df['extracted_location'].value_counts().head(5)
print("Top 5 crisis-related locations:")
print(top_locations)

Top 5 crisis-related locations:
extracted_location
Delhi        1
Mumbai       1
Bangalore    1
Chennai      1
Kolkata      1
Name: count, dtype: int64
